In [4]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 748.6 kB/s eta 0:00:0000:0100:01


In [ ]:
!pip install pandas 

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
from time import time

In [82]:
def database_connect(user, password, host, port, db):
    engine = create_engine(
        f'postgresql://{user}:{password}@{host}:{port}/{db}')
    engine.connect()
    return engine

def fetch_data(url, csv_name): 
    os.system(f"wget {url} -O {csv_name}")

def batch_load_data(engine, csv_file, table_name): 
    df_iter = pd.read_csv(csv_file, iterator=True, chunksize=100000)
    df = next(df_iter)
    
    df.InvoiceDate = pd.to_datetime(df.InvoiceDate)
    df.head(n=0).to_sql(name=table_name, con=engine, if_exists='replace', index=True, index_label='Id')
    df.to_sql(name=table_name, con=engine, if_exists='append', index=True, index_label='Id')

    elapsed_time = -1
    try:
        t_start = time()
        chuncks_cnt = 0
        
        while True:
            chuncks_cnt += 1
            chunck_start = time()
        
            df = next(df_iter)
            df.InvoiceDate = pd.to_datetime(df.InvoiceDate)
            df.to_sql(name=table_name, con=engine, if_exists='append', index=True, index_label='Id')
            print(f'Inserted chunck {chuncks_cnt} in {time()-chunck_start:.3}s')
    except:
        elapsed_time = time() - t_start 
        print(f'Finished ingestion into table: {table_name}')
        print(f'Number of chuncks: {chuncks_cnt}')
        print(f'Time taken: {elapsed_time:.3}s')
        
    return elapsed_time
    

In [83]:
db_engine = database_connect('root', 'root', 'postgres-db', '5432', 'RetailDB')

In [84]:
batch_load_data(db_engine, 'online_retail.csv', 'online_retail')

Inserted chunck 1 in 3.53s
Inserted chunck 2 in 3.82s
Inserted chunck 3 in 3.84s
Inserted chunck 4 in 3.47s
Inserted chunck 5 in 1.79s
Finished ingestion into table: online_retail
Number of chuncks: 6
Time taken: 16.4s


16.44627833366394

In [69]:
df = pd.read_csv('online_retail.csv')

In [88]:
df.shape

(541909, 8)

In [53]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [54]:
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [62]:
print(pd.io.sql.get_schema(df, 'schema'))

CREATE TABLE "schema" (
"InvoiceNo" TEXT,
  "StockCode" TEXT,
  "Description" TEXT,
  "Quantity" INTEGER,
  "InvoiceDate" TIMESTAMP,
  "UnitPrice" REAL,
  "CustomerID" REAL,
  "Country" TEXT,
  "ID" INTEGER
)


In [56]:
df['ID'] = df.index

In [61]:
df.set_index('ID')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
ID,,,,,,,,
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
